In [1]:
import pandas as pd

# Assuming you have the relevant data available as pandas DataFrames or CSV files,
# you can start by creating the required DataFrames for each step in the funnel.


downloads_df = pd.read_csv('metrocar_data/app_downloads.csv')

signups_df = pd.read_csv('metrocar_data/signups.csv')

ride_requests_df = pd.read_csv('metrocar_data/ride_requests.csv')

accepted_requests_df = ride_requests_df[ride_requests_df['accept_ts'].notnull()]

completed_rides_df = accepted_requests_df[accepted_requests_df['cancel_ts'].isnull()]

reviews_df = pd.read_csv('metrocar_data/reviews.csv')

transactions_df = pd.read_csv('metrocar_data/transactions.csv')

pre_ride_df = downloads_df.merge(signups_df, left_on='app_download_key', right_on='session_id', how='inner')

filtered_transactions_df = transactions_df[transactions_df['charge_status'] == 'Approved']

payment_df = ride_requests_df.merge(filtered_transactions_df, on='ride_id', how='inner')

#payment_df = payment_df[payment_df['charge_status'] == 'Approved']



/var/folders/1m/hvnyjbcx2rxfb9d__rb_m7h00000gp/T/ipykernel_12644/3059708972.py:11: DtypeWarning: Columns (4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  ride_requests_df = pd.read_csv('metrocar_data/ride_requests.csv')


,user_id,count
0,100002,1
1,100004,1
2,100007,1
3,100008,1
4,100010,1
...,...,...
6228,117614,1
6229,117615,1
6230,117617,1
6231,117621,1


In [ ]:
payment_df

In [7]:
transactions_df[transactions_df['charge_status'] == 'Approved']

,transaction_id,ride_id,purchase_amount_usd,charge_status,transaction_ts
0,10000000,3000000,13.55,Approved,2021-03-28 19:11:00
1,10000001,3000001,27.77,Approved,2021-11-10 16:59:00
2,10000002,3000002,21.84,Approved,2021-09-08 21:03:00
3,10000003,3000004,26.86,Approved,2021-05-28 09:21:00
4,10000004,3000005,21.72,Approved,2021-12-01 16:59:00
...,...,...,...,...,...
223646,10223646,3385468,27.23,Approved,2022-01-30 09:21:00
223647,10223647,3385469,25.06,Approved,2021-11-12 17:57:00
223648,10223648,3385472,23.29,Approved,2021-10-31 18:24:00
223649,10223649,3385473,18.88,Approved,2021-11-05 10:42:00


In [4]:
# Perform funnel analysis

steps = [
    ('App Download', downloads_df, 'app_download_key'),
    ('Signup', signups_df, 'user_id'),
    ('Request Ride', ride_requests_df, 'user_id'),
    ('Driver Acceptance', accepted_requests_df, 'user_id'),
    ('Completed Ride', completed_rides_df, 'user_id'),
    ('Completed Payment', payment_df, 'user_id'),
    ('Completed Review', reviews_df, 'user_id')
]

funnel_data = []
previous_count = None

for step, df, id_column in steps:
    count = df[id_column].nunique() if id_column else len(df)
    conversion = count / previous_count * 100 if previous_count is not None else None
    drop_off = 100 - conversion if conversion is not None else None
    
    funnel_data.append({
        'Step': step,
        'Count': count,
        'Previous Count': previous_count,
        'Conversion': conversion,
        'Drop-off': drop_off
    })
    
    previous_count = count

# Convert the funnel data to a pandas DataFrame for further analysis or visualization
funnel_df = pd.DataFrame(funnel_data)

# Display the funnel DataFrame
print(funnel_df)


                Step  Count  Previous Count  Conversion   Drop-off
0       App Download  23608             NaN         NaN        NaN
1             Signup  17623         23608.0   74.648424  25.351576
2       Request Ride  12406         17623.0   70.396641  29.603359
3  Driver Acceptance  12278         12406.0   98.968241   1.031759
4     Completed Ride   6233         12278.0   50.765597  49.234403
5  Completed Payment   6233          6233.0  100.000000   0.000000
6   Completed Review   4348          6233.0   69.757741  30.242259


/var/folders/1m/hvnyjbcx2rxfb9d__rb_m7h00000gp/T/ipykernel_12445/3467492101.py:3: DtypeWarning: Columns (4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  ride_requests = pd.read_csv('metrocar_data/ride_requests.csv')
